## Her skal vi rette på feilene som vi lagde eller allerede har i dataene våre

Først kan vi begynne med df2_errors, der vi har uregelmessigheter i dataen, med både "null" entries og uteliggere. 

Her er funksjoner som kan brukes til å rette opp feilene: 

In [10]:
import pandas as pd
import numpy as np

# Leser filen df2_errors.json
df2_errors=pd.read_json('../data/df2_errors.json')

df2_errors['referenceTime']=pd.to_datetime(df2_errors['referenceTime'],unit='ms')

def håndter_mangler(df,column,method='mean',degree=2):
    if method=='median':
        df[column]=df[column].fillna(df[column].median())
    elif method == 'interpolate':
        df[column] = df[column].interpolate(method='polynomial',order=degree)
    elif method == 'ffill':
        df[column] = df[column].fillna(method='ffill')
    elif method == 'bfill':
        df[column] = df[column].fillna(method='bfill')
    else: 
        df[column] = df[column].fillna(df[column].mean())
    return df

def håndter_uteliggere(df,column,method='mean',degree=2):
    øvre_grense=45
    nedre_grense=-20

    df[column]=np.where((df[column]<nedre_grense) | (df[column]>øvre_grense),np.nan,df[column])
    return håndter_mangler(df,column,method='mean')

df2_errors=håndter_mangler(df2_errors,'mean(relative_humidity P1D)',method='interpolate',degree=1)
df2_errors=håndter_uteliggere(df2_errors,'mean(air_temperature P1D)')

df2_errors.to_json('../data/df2_fikset.json',indent=4)


Videre så må vi fikse på luftkvalitetsdataen, som har en del nullverdier. 

In [14]:
df1_data=pd.read_json('../data/df1_data.json')

df1_data['date']=pd.to_datetime(df1_data['date'],unit='ms')

df1_data=håndter_mangler(df1_data,'CO',method='interpolate',degree=1)
df1_data=håndter_mangler(df1_data,'NO',method='interpolate',degree=1)
df1_data=håndter_mangler(df1_data,'NO2',method='interpolate',degree=1)
df1_data=håndter_mangler(df1_data,'PM10',method='interpolate',degree=1)
df1_data=håndter_mangler(df1_data,'NOx',method='interpolate',degree=1)
df1_data=håndter_mangler(df1_data,'PM2.5',method='interpolate',degree=1)

df1_data.to_json('../data/df1_fikset.json',indent=4)
